In [1]:
!pip install faiss-cpu openai groq langchain langchain-groq tiktoken cohere chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjso

In [3]:
import os 

os.environ["GROQ_API_KEY"] = "enter api key"
os.environ["OPENAI_API_KEY"] = "enter api key"

### Load docs + split

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://paulgraham.com/greatwork.html")
docs = loader.load()

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
splitted_docs = r_splitter.split_documents(docs)

### Loading embeddings

In [22]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Index

In [23]:
from langchain_community.vectorstores import FAISS

vectoe = FAISS.from_documents(
    splitted_docs,
    embeddings
)

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key = "api key",
    model_name = 'mixtral-8x7b-32768'
)

### Prompt + Chain

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following quesion based on the retrieved context.
    Think step by step.
    You will get a reward if your answers were helpful.
    <context>
    {context}
    </context>
    
    Question: {input}
    """
)

llm_chain = create_stuff_documents_chain(
    llm, 
    prompt
)

### Retrieval + Testing

In [26]:
from langchain.chains import create_retrieval_chain

retriever = vectoe.as_retriever()
retrieval_chain = create_retrieval_chain(
    retriever, 
    llm_chain
)

In [28]:
prompt = "What does the author say doing great work?"

response = retrieval_chain.invoke({"input": prompt})
response['answer']

"The author suggests that doing great work involves several factors. Firstly, it is about doing something important and significant, and doing it exceptionally well, to the point of expanding people's ideas of what's possible. However, the author notes that there is no precise threshold for importance, and it's a matter of degree, often hard to judge at the time.\n\nSecondly, doing great work often involves noticing anomalies in everyday life and having new ideas about nontrivial things. It requires a certain level of optimism and earnestness, being willing to take the risk of telling people your ideas and potentially looking like a fool.\n\nLastly, the author suggests focusing on developing one's interests and trying to do something amazing, leaving it to future generations to say if you succeeded. Overall, doing great work involves a combination of hard work, optimism, noticing anomalies, and a willingness to take risks and be earnest in one's pursuits."